### Algorithm:

1. Search the field replacing keywords in the URL

2. Get all the names and URLs from the page

3. Check if the keyword appears in each URL name

4. If the keyword appears in the URL name, add the name and URL to the list

5. Get 'next' URL and repeat the process until the keyword does not appear in the URL name

6. Get all the pages of the urls collected and get the
date filed for each URL

7. Sort the list by date filed

8. Remove the entries more than 30 days old

9. Print the list of names and URLs


In [56]:
import pandas as pd
import bs4
import requests

In [57]:
names_and_links = []
name = "semiconductor"
header_link = 'https://search.sunbiz.org/'
r = requests.get(f'{header_link}Inquiry/CorporationSearch/SearchResults/EntityName/{name}/Page1?searchNameOrder={name.upper()}')
r.raise_for_status()

soup = bs4.BeautifulSoup(r.text, 'html.parser')


print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-PFV0FXXNRV">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'G-PFV0FXXNRV');
  </script>
  <script type="text/javascript">
   var _gaq = _gaq || [];
        _gaq.push(['_setAccount', 'UA-3263504-14']);
        _gaq.push(['_setDomainName', 'sunbiz.org']);
        _gaq.push(['_trackPageview']);

        (function () {
            var ga = document.createElement('script');
            ga.type = 'text/javascript';
            ga.async = true;
            ga.src = ('https:' == document.location.protocol ? 'https://ssl' : 'http://www') + '.google-analytics.com/ga.js';
            var s = document.getElementsByTagName('script')[0];
            s.parentNode.insertBefore(ga, s);
        })();
  </script>
  <meta content="te

In [58]:
search_results = soup.find_all(attrs = {'id': 'search-results'})
names_and_links = search_results[0].find_all('tr')
names_and_links.extend([(names_and_links[i].td.text, names_and_links[i].a['href']) for i in range(1, len(names_and_links))])

print(names_and_links, len(names_and_links), sep = '\n')

[('SEMICONDUCTOR COMPONENTS INDUSTRIES, LLC', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=Initial&searchNameOrder=SEMICONDUCTORCOMPONENTSINDUSTR%20M990000011350&aggregateId=forl-m99000001135-b9609849-653e-4fa0-b62f-65d95d0f8dd9&searchTerm=semiconductor&listNameOrder=SEMICONDUCTORCOMPONENTSINDUSTR%20M990000011350'), ('SEMICONDUCTOR COMPONENT SUBSTRATES CORPORATION', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=Initial&searchNameOrder=SEMICONDUCTORCOMPONENTSUBSTRAT%208291480&aggregateId=forp-829148-93f516de-11c1-4da7-9369-ac2d1a69a0d1&searchTerm=semiconductor&listNameOrder=SEMICONDUCTORCOMPONENTSINDUSTR%20M990000011350'), ('SEMICONDUCTOR CONSULTING LLC', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=Initial&searchNameOrder=SEMICONDUCTORCONSULTING%20L210004784370&aggregateId=flal-l21000478437-f899bc99-a138-433d-9d4d-d5a7407dfe21&searchTerm=semiconductor&listNameOrder=SEMICO

In [61]:
def check_name(keyword, namelink_list):
    length = len(namelink_list)
    status = []
    for idx in range(length):
        if keyword.lower() in namelink_list[idx][0].lower():
            status.append((namelink_list[idx][0], True))
        else:
            status.append((namelink_list[idx][0], False))
    return status
checked_list = check_name(name, names_and_links)

In [62]:
checked_dt = pd.DataFrame(checked_list, columns = ['Name', 'Match'])

In [64]:
if (checked_dt.Match.value_counts()[0]/len(checked_dt)) > 0.8:
    print('Next Page')
else:
    print('Last Page')

Next Page


/tmp/ipykernel_13246/1481764436.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (checked_dt.Match.value_counts()[0]/len(checked_dt)) > 0.8:


In [65]:
next_button = soup.find_all('div', attrs = {'class': 'navigationBarPaging'})
next_button_link = next_button[0].find_all('a')[-1]['href'].lstrip('/')

In [66]:
r = requests.get(header_link + next_button_link)
r.raise_for_status()

soup = bs4.BeautifulSoup(r.text, 'html.parser')


print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-PFV0FXXNRV">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'G-PFV0FXXNRV');
  </script>
  <script type="text/javascript">
   var _gaq = _gaq || [];
        _gaq.push(['_setAccount', 'UA-3263504-14']);
        _gaq.push(['_setDomainName', 'sunbiz.org']);
        _gaq.push(['_trackPageview']);

        (function () {
            var ga = document.createElement('script');
            ga.type = 'text/javascript';
            ga.async = true;
            ga.src = ('https:' == document.location.protocol ? 'https://ssl' : 'http://www') + '.google-analytics.com/ga.js';
            var s = document.getElementsByTagName('script')[0];
            s.parentNode.insertBefore(ga, s);
        })();
  </script>
  <meta content="te